# <p style="text-align: center"> Limpando Dados do OpenStreetMap da Região Metropolitana de São Paulo </p>  

<p style="text-align: center"> [DW-OSM-RMSP.ipynb](http://nbviewer.jupyter.org/github/vsvasconcelos/-DW-OSM-RMSP/blob/master/DW-OSM-RMSP.ipynb)
---
### <p style="text-align: center"> Vagner Sanches Vasconcelos

## Resumo: </p>
Neste projeto são utilizados técnicas de tratamento e análise de dados da base do projeto OpenStreetMap especificamente da Região Metropolitana de São Paulo (RMSP); para isso foi utilzado a linguagem de programação Python e o banco de dados MongoDB.

# 1.0) INTRODUÇÃO </p>
Basicamente o processo de análise de dados parte de uma questão ou um problema ao qual deseja-se a resposta; contudo, para chegar a ela, antes é necessário passar pela fase de preparação ou tratamento dos dados (*data wrangling* ou *data munging*), que envolve: obtenção (*gathering*), extração (*extracting*), limpeza (*cleaning*) e armazenamento (storing) dos dados. Após a fase de preparação ocorre efetivamente a análise dos dados, onde estes são explorados de forma a buscar as respostas as questões/problemas que deseja-se responder; ao fim desta análise, conclusões são desenvolvidas e finalmente apresentadas em relatórios.</p>  Segundo o [NYT](https://www.nytimes.com/2014/08/18/technology/for-big-data-scientists-hurdle-to-insights-is-janitor-work.html?_r=0), o processo de preparação dos dados é uma tarefa que ocupa entre 50 e 80% do tempo do analista.

## 1.1) Questões de Pesquisa </p>  
Todas as questões abaixo se referem aos dados que serão carregado no banco de dados MongoDB.

### 1.1.1) Qual o tamanho da base?
### 1.1.2) Qual o número de usuários únicos na base?
### 1.1.3) Quais os 3 usuário com maior número de contribuições?
### 1.1.4) Qual o número de nós e caminhos encontrado na base?
### 1.1.5) Qual o número de nós do tipo "Ponto de Ônibus"?

# 2.0)  TRATAMENTO DOS DADOS </p>  

## 2.1) Obtenção dos Dados </p>  
Os dados para realização deste projeto foram obtidos da fonte secundária [OpenStreetMap](http://www.openstreetmap.com.br/), que é um projeto de produção colaborativa de dados geo-espaciais abertos, no qual qualquer pessoa pode editar o mapa e os dados são redistribuídos sob a licença [ODbL](http://www.openstreetmap.org/copyright?locale=pt-BR). </p>  
A [RMSP](https://pt.wikipedia.org/wiki/Regi%C3%A3o_Metropolitana_de_S%C3%A3o_Paulo) reúne 39 municípios do estado de São Paulo, sendo a maior região metropolitana do Brasil, e uma das dez mais populosas do mundo.
![RMSP](https://upload.wikimedia.org/wikipedia/commons/thumb/5/53/SaoPaulo_RM_SaoPaulo.svg/280px-SaoPaulo_RM_SaoPaulo.svg.png)</p>  

## 2.2) Extração dos Dados </p>  
A delimitação da RMSP pode ser acessada no [OpenStreetMap](https://www.openstreetmap.org/relation/2661855); contudo, o portal [MapZen](http://wiki.openstreetmap.org/wiki/Mapzen) já possui a base de dados dela pré-selecionada e pronta para download, em vários formatos, sendo o utilizado neste trabalho o *Raw OpenStreetMap datasets* (__XML__); os dados do MapZen são oriundos do OpenStreetMap. Neste [link](https://mapzen.com/data/metro-extracts/metro/sao-paulo_brazil/) estão os dados da RMSP, num arquivo compactado (bz2) de 50MB, que após descompactado gera o arquivo *sao-paulo_brazil.osm* com 764MB. A documentação deste arquivo pode ser acessada [aqui](http://wiki.openstreetmap.org/wiki/OSM_XML).

## 2.3) Limpeza dos Dados </p>  

Esta etapa foi realizada seguindo as melhores práticas de limpeza de dados (blueprint) conforme [Bradshaw](http://nbviewer.jupyter.org/github/vsvasconcelos/-DW-OSM-RMSP/blob/master/blueprint.ipynb).

### 2.3.1) Auditória dos Dados

Conforme já apresentado, o conjunto de dados deste trabalho possui 764MB; para facilitar o processo de auditória, começaremos o trabalho analisando uma pequena amosta da base, assim a iteração na investigação será feita mais rapidamente; após validado este processo, ele será aplicado em toda a base.</p>  A amostra da base foi obtida por meio do programa [AmostraBase.ipynb](http://nbviewer.jupyter.org/github/vsvasconcelos/-DW-OSM-RMSP/blob/master/AmostraBase.ipynb), sendo que seu tamanho ficou em 77,5MB. </p>    
Inicialmente serão auditados os tipos de vias, tais como: rua; avenida; alameda; praça, etc. Conforme [documentação](http://wiki.openstreetmap.org/wiki/Key:addr), os elementos *node* e *way* utilizam a tag *addr:street* com a finalidade de registrar o tipo de via com seu respectivo nome. O trecho abaixo da base de
dados ilustra o exemplo da *Rua João Boemer*.

In [ ]:
<node changeset="45082900" id="4602264559" lat="-23.5312944" lon="-46.6132484" timestamp="2017-01-11T18:12:06$
                <tag k="addr:city" v="São Paulo" />
                <tag k="addr:housenumber" v="1117" />
                <tag k="addr:postcode" v="03018-000" />
                <tag k="addr:street" v="Rua João Boemer" />
</node>

Para obtenção dos nomes das vias e seus respectivos tipos, foi utilizado o módulo de expressões regulares [(re)](https://docs.python.org/2/library/re.html) do Python; sendo este instalado com o seguinte comando: __pip install regex__. </p>   O programa [TiposDeVias.ipynb](http://nbviewer.jupyter.org/github/vsvasconcelos/-DW-OSM-RMSP/blob/master/TiposDeVias.ipynb) busca os tipos de vias encontrados na base bem como suas quantidades. </p>  Conforme a saída do programa *TiposDeVias.ipynb* - apresentada abaixo - na amostra da base foram encontrados 29 tipos de via, distribuidos em 2353 vias, sendo *Rua* o tipo de maior frequência com 1530 registros, seguido por *Avenida* com 694 registros. Contudo, temos por exemplo, os registros *rua, R. e Av.* que respectivamente devem ser os dois registros mais encontrados, mas por desejo dos usuários foram lançados desta forma.

Saída do programa [TiposDeVias.ipynb](http://nbviewer.jupyter.org/github/vsvasconcelos/-DW-OSM-RMSP/blob/master/TiposDeVias.ipynb) (Nome da via: Quantidade de registros): </p>  Acesso: 1 , Alameda: 26, Alfonso: 1, Av.: 2, Avenida: 694, Calçadão: 1, Complexo: 1, Conselheiro: 1, Dr.: 1, Estrada: 27, Guarara: 1, Ladeira: 1, Largo: 8, Marginal: 1, Passeio: 1, Praça: 21, R.: 3, Rocha: 1, Rodoanel: 1, Rodovia: 11, Rua: 1530, rua: 1, sao: 1, Tavares: 1, Travessa: 9, Via: 3, Viaduto: 1, Viela: 1, Vila: 2

Fazendo buscas manuais pelos tipos de vias na base de dados, ex.*Alfonso* - conforme trecho do arquivo abaixo - se observa que o usuário não lançou um tipo de via para esta; isso aconteceu para outros casos, tais como: *Tavares, Rocha, Guarara, Dr. e Conselheiro*

In [ ]:
                <tag k="name" v="Q Pizza" />
                <tag k="amenity" v="fast_food" />
                <tag k="addr:city" v="São Paulo" />
                <tag k="addr:street" v="Alfonso Bovero" />
                <tag k="addr:housenumber" v="724" />

Com os tipos de via reconhecidos, o próximo passo foi auditá-los, identificando todos os tipos de via estranhos presente na base; o programa [AuditaTiposDeVias.ipynb](http://nbviewer.jupyter.org/github/vsvasconcelos/-DW-OSM-RMSP/blob/master/AuditaTiposDeVias.ipynb) realiza essa identificação.
Abaixo segue um trecho da saída deste programa:

In [ ]:
{u'Av.': set(['Av. Dn. Ana Costa', u'Av. Francisco N\xf3brega Barbosa']),
 u'Cal\xe7ad\xe3o': set([u'Cal\xe7ad\xe3o Amalia Sestine']),
 u'Complexo': set([u'Complexo Vi\xe1rio Engenheiro Job Shuji Nogami']),
 'Marginal': set(['Marginal Pinheiros local']),
 'Passeio': set(['Passeio Itaparica']),
 u'Pra\xe7a': set([u'Pra\xe7a Bom Jesus de Piraporinha',
                   u'Pra\xe7a IV Centen\xe1rio',
                   u'Pra\xe7a Mau\xe1',
                   u'Pra\xe7a Miguel Ortega',
                   ...
                   u'Pra\xe7a do Oceoan\xf3grafo']),
 'R.': set(['R. Bergamo']),
 'Rodoanel': set(['Rodoanel Mario Covas Leste']),
 'Via': set(['Via Lateral (Norte)',
             'Via Parque',
             u'Via de Acesso Jo\xe3o de Goes'])}

A próxima auditoria realizada foi em relação a todos os tags; o programa [AuditaTags.ipynb](http://nbviewer.jupyter.org/github/vsvasconcelos/-DW-OSM-RMSP/blob/master/AuditaTags.ipynb), examina cada um deles verificando e classificando eventuais desvios.</p>    <p style="text-align: center"> __Classificação__: </p>   
*lower*: Para tags válidas com somente letras minúsculas;  </p>  
*lower_colon*: Para tags válidas com dois-pontos em seus nomes;  </p>    
*problemchars*: Para tags com caracteres problemáticos; e </p>    
*other*: Para todos os casos que não se enquadram nas 3 outras categórias.</p>   
Conforme a saída deste programa, apresenta abaixo, 121 tags apresentaram desvios. 

In [ ]:
{'lower': 139602, 'lower_colon': 10452, 'other': 120, 'problemchars': 1}

Ainda com relação aos tags, o programa [ContagemDosTags.ipynb](http://nbviewer.jupyter.org/github/vsvasconcelos/-DW-OSM-RMSP/blob/master/ContagemDosTags.ipynb) realiza a identificação e a contagem destes. </p>  Abaixo segue a saída do programa, com os tags identificados bem como suas quantidades.

{'member': 6841,
 'nd': 458477,
 'node': 338061,
 'osm': 1,
 'relation': 1010,
 'tag': 150175,
 'way': 46939}

Agora serão auditados os usuários da base, o programa [AuditaUsuarios.ipynb](http://nbviewer.jupyter.org/github/vsvasconcelos/-DW-OSM-RMSP/blob/master/AuditaUsuarios.ipynb) reconhece as identificações dos usuários na base.
Abaixo é apresentado o trecho inicial da saída do programa, a qual apresenta que foram identificados 1233 usuários e mostrando suas identificações (uid).

In [ ]:
1233
set(['102069', '1385426', '1886529', '1866562', '2678586', '967832', '152074', '1425865', '2032261', '526341', ...

### 2.3.2) Plano de Limpeza dos Dados </p> 


### 2.3.3) Execução do Plano de Limpeza dos Dados </p>  

## 2.4) Armazenamento dos Dados </p>  

O armazenamento dos dados foi realizado no [MongoDB](https://pt.wikipedia.org/wiki/MongoDB), que é um banco de dados orientados a documentos do tipo [JSON](https://pt.wikipedia.org/wiki/JSON); assim, para subir os dados do OpenStreetMap, que são arquivos XML, para o MongoDB é necessário realizar a conversão de XML para JSON, o que foi realizado com o programa [Xml2Json.ipynb](http://nbviewer.jupyter.org/github/vsvasconcelos/-DW-OSM-RMSP/blob/master/Xml2Json.ipynb). </p> 
A saída deste programa foi um arquivo JSON de 86,6MB, isto é, maior que o arquivo XML que o originou (77,5MB). Abaixo é apresentado uma linha deste arquivo de saída.

In [ ]:
{"created": {"changeset": "16953747",
             "user": "fbello",
             "version": "3", 
             "uid": "79543", 
             "timestamp": "2013-07-14T19:50:01Z"},
 "type": "node",
 pos": [-23.5594684, -46.7072388],
 "id": "573640"} 

O arquivo JSON foi carregado no MongoDB utilizando o comando __*mongoimport -d sample-osm -c rmsp --file sample.json*__ - no prompt do sistema operacional - no qual: *sample-osm*, é nome da base da dados; e *rmsp* é o nome da coleção.

# 3)  ANÁLISE DOS DADOS </p>  

### Qual o tamanho da base?
*__show dbs__*
>  (0.203GB)

### Qual o número de documentos da base?
*__db.rmsp.find().count()__*
>  385000

### Qual o número de usuários únicos na base?
*__db.rmsp.distinct('created.user').length__*
> 1233

### Quais os 3 usuários com maior número de contribuições?

In [ ]:
db.rmsp.aggregate([{"$group": {"_id":"$created.user","count":{"$sum":1}}},{"$sort":{"count":-1}},{"$limit":3}

>      
    { "_id" : "Bonix-Mapper", "count" : 171080 }
    { "_id" : "AjBelnuovo", "count" : 21330 }
    { "_id" : "cxs", "count" : 19876 }

### Qual o número de nós e caminhos encontrado na base?
Número de nós (node):

    db.rmsp.find({"type":"node"}).count() 
> 338059

Número de caminhos (way):

     db.rmsp.find({"type":"way"}).count()
> 46935

### Qual o número de nós com o nome de  "Ponto de Ônibus"?

### Quais as 10 [amenities](http://wiki.openstreetmap.org/wiki/Key:amenity) que mais apareceram?

    db.rmsp.aggregate([{"$match":{"amenity":{"$exists":1}}},{"$group":{"_id":"$amenity","count":{"$sum":1}}},{"$sort":{"count":-1}},{"$limit":10}])

>   
    { "_id" : "parking", "count" : 303 }
    { "_id" : "fuel", "count" : 186 }
    { "_id" : "restaurant", "count" : 128 }
    { "_id" : "school", "count" : 117 }
    { "_id" : "bank", "count" : 103 }
    { "_id" : "fast_food", "count" : 71 }
    { "_id" : "place_of_worship", "count" : 69 }
    { "_id" : "pharmacy", "count" : 52 }
    { "_id" : "pub", "count" : 34 }
    { "_id" : "hospital", "count" : 30 }

# 4.0) CONCLUSÕES

Problemas encontrados no seu mapa </p>   Visão geral dos Dados  </p>     
Outras ideias em relação aos conjuntos de dados  